In [74]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import seaborn as sns
import os
import pandas as pd
from PIL import Image

In [75]:
# Inception Module Block (Feature Extraction Block)
class InceptionBlock(nn.Module):
    def __init__(self, in_channels, f1, f2, f3, f4):
        """
        Inception block that combines feature maps from different kernel sizes.
        
        Args:
            in_channels: Number of input channels
            f1: Number of filters for 1x1 convolution
            f2: Number of filters for 3x3 convolution
            f3: Number of filters for 5x5 convolution
            f4: Number of filters for max pooling path
        """
        super(InceptionBlock, self).__init__()
        
        # 1x1 convolution branch
        self.branch1 = nn.Sequential(
            nn.Conv2d(in_channels, f1, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(f1),
            nn.ReLU(inplace=True)
        )
        
        # 3x3 convolution branch
        self.branch2 = nn.Sequential(
            nn.Conv2d(in_channels, f2, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(f2),
            nn.ReLU(inplace=True)
        )
        
        # 5x5 convolution branch
        self.branch3 = nn.Sequential(
            nn.Conv2d(in_channels, f3, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(f3),
            nn.ReLU(inplace=True)
        )
        
        # Max pooling branch
        self.branch4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            nn.Conv2d(in_channels, f4, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(f4),
            nn.ReLU(inplace=True)
        )
    
    def forward(self, x):
        branch1 = self.branch1(x)
        branch2 = self.branch2(x)
        branch3 = self.branch3(x)
        branch4 = self.branch4(x)
        
        # Concatenate all branches
        outputs = torch.cat([branch1, branch2, branch3, branch4], dim=1)
        return outputs

In [76]:
# Asbestos Detection CNN with Inception-based Architecture
class AsbestosDetectionCNN(nn.Module):
    def __init__(self, num_classes=2):
        """
        CNN architecture based on Inception-Net for asbestos roof detection.
        Input: 100x100x3 RGB images
        """
        super(AsbestosDetectionCNN, self).__init__()
        
        # Initial convolutional block
        self.conv_block = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(64)
        )
        
        # Four Inception feature extraction blocks
        # Block 1: 64 -> 256 channels (64+64+64+64)
        self.inception1 = InceptionBlock(64, 64, 64, 64, 64)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout1 = nn.Dropout2d(p=0.55)
        
        # Block 2: 256 -> 512 channels (128+128+128+128)
        self.inception2 = InceptionBlock(256, 128, 128, 128, 128)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout2 = nn.Dropout2d(p=0.55)
        
        # Block 3: 512 -> 1024 channels (256+256+256+256)
        self.inception3 = InceptionBlock(512, 256, 256, 256, 256)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout3 = nn.Dropout2d(p=0.55)
        
        # Block 4: 1024 -> 1024 channels (256+256+256+256)
        self.inception4 = InceptionBlock(1024, 256, 256, 256, 256)
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout4 = nn.Dropout2d(p=0.55)
        
        # Calculate flattened size: 100x100 -> 50x50 -> 25x25 -> 12x12 -> 6x6
        # After 4 pooling layers: 6x6x1024 = 36864
        self.flatten_size = 6 * 6 * 1024
        
        # Two fully connected blocks
        # FC Block 1
        self.fc1 = nn.Linear(self.flatten_size, 1024)
        self.bn_fc1 = nn.BatchNorm1d(1024)
        self.relu_fc1 = nn.ReLU(inplace=True)
        self.dropout_fc1 = nn.Dropout(p=0.50)
        
        # FC Block 2
        self.fc2 = nn.Linear(1024, 1024)
        self.bn_fc2 = nn.BatchNorm1d(1024)
        self.relu_fc2 = nn.ReLU(inplace=True)
        self.dropout_fc2 = nn.Dropout(p=0.50)
        
        # Output layer
        self.output = nn.Linear(1024, num_classes)
    
    def forward(self, x):
        # Initial conv block
        x = self.conv_block(x)
        
        # Inception blocks with pooling and dropout
        x = self.inception1(x)
        x = self.pool1(x)
        x = self.dropout1(x)
        
        x = self.inception2(x)
        x = self.pool2(x)
        x = self.dropout2(x)
        
        x = self.inception3(x)
        x = self.pool3(x)
        x = self.dropout3(x)
        
        x = self.inception4(x)
        x = self.pool4(x)
        x = self.dropout4(x)
        
        # Flatten
        x = x.view(x.size(0), -1)
        
        # FC blocks
        x = self.fc1(x)
        x = self.bn_fc1(x)
        x = self.relu_fc1(x)
        x = self.dropout_fc1(x)
        
        x = self.fc2(x)
        x = self.bn_fc2(x)
        x = self.relu_fc2(x)
        x = self.dropout_fc2(x)
        
        # Output with softmax
        x = self.output(x)
        return F.softmax(x, dim=1)

In [77]:
# Custom Dataset class for asbestos detection
class AsbestosDataset(Dataset):
    def __init__(self, images, labels, standardize=True):
        """
        Dataset for asbestos roof images.
        
        Args:
            images: numpy array of shape (N, 100, 100, 3)
            labels: numpy array of shape (N,) with binary labels (0 or 1)
            standardize: whether to apply feature-wise standardization
        """
        self.images = images
        self.labels = labels
        self.standardize = standardize
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        image = self.images[idx].astype(np.float32)
        label = self.labels[idx]
        
        # Feature-wise standardization (per image)
        if self.standardize:
            mean = image.mean(axis=(0, 1), keepdims=True)
            std = image.std(axis=(0, 1), keepdims=True) + 1e-7
            image = (image - mean) / std
        
        # Convert to tensor and change from HWC to CHW format
        image = torch.from_numpy(image).permute(2, 0, 1)
        label = torch.tensor(label, dtype=torch.long)
        
        return image, label

In [78]:
# Learning rate scheduler
class ReduceLROnPlateau:
    def __init__(self, optimizer, patience=5, factor=0.001, min_lr=1e-6):
        self.optimizer = optimizer
        self.patience = patience
        self.factor = factor
        self.min_lr = min_lr
        self.best_loss = float('inf')
        self.wait = 0
    
    def step(self, val_loss):
        if val_loss < self.best_loss:
            self.best_loss = val_loss
            self.wait = 0
        else:
            self.wait += 1
            if self.wait >= self.patience:
                self._reduce_lr()
                self.wait = 0
    
    def _reduce_lr(self):
        for param_group in self.optimizer.param_groups:
            old_lr = param_group['lr']
            new_lr = max(old_lr - self.factor, self.min_lr)
            param_group['lr'] = new_lr
            if new_lr != old_lr:
                print(f'Reducing learning rate from {old_lr:.6f} to {new_lr:.6f}')

class StepLRScheduler:
    def __init__(self, optimizer, step_size=10, gamma=0.85):
        self.optimizer = optimizer
        self.step_size = step_size
        self.gamma = gamma
        self.last_epoch = 0
    
    def step(self, epoch):
        if (epoch + 1) % self.step_size == 0:
            for param_group in self.optimizer.param_groups:
                old_lr = param_group['lr']
                new_lr = old_lr * self.gamma
                param_group['lr'] = new_lr
                print(f'Epoch {epoch+1}: Reducing learning rate from {old_lr:.6f} to {new_lr:.6f}')
        self.last_epoch = epoch

In [79]:
# Training function
def train_model(model, train_loader, val_loader, num_epochs=128, learning_rate=0.0015, device='cpu', save_dir='./models'):
    """
    Train the asbestos detection model.
    
    Args:
        model: AsbestosDetectionCNN model
        train_loader: DataLoader for training data
        val_loader: DataLoader for validation data
        num_epochs: Number of training epochs (default: 128)
        learning_rate: Initial learning rate (default: 0.0015)
        device: Device to train on ('cpu' or 'cuda')
        save_dir: Directory to save model checkpoints
    
    Returns:
        Dictionary containing training history
    """
    Path(save_dir).mkdir(parents=True, exist_ok=True)
    
    model = model.to(device)
    
    # Binary cross-entropy loss
    criterion = nn.CrossEntropyLoss()
    
    # Adam optimizer
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    # Learning rate schedulers
    step_scheduler = StepLRScheduler(optimizer, step_size=10, gamma=0.85)
    plateau_scheduler = ReduceLROnPlateau(optimizer, patience=5, factor=0.001)
    
    # Training history
    history = {
        'train_loss': [],
        'train_acc': [],
        'val_loss': [],
        'val_acc': [],
        'learning_rates': []
    }
    
    best_val_loss = float('inf')
    best_model_path = None
    
    print(f"Training on device: {device}")
    print(f"Total epochs: {num_epochs}, Batch size: {train_loader.batch_size}")
    print("=" * 70)
    
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        
        for batch_idx, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            _, predicted = outputs.max(1)
            train_total += labels.size(0)
            train_correct += predicted.eq(labels).sum().item()
        
        avg_train_loss = train_loss / len(train_loader)
        train_accuracy = 100. * train_correct / train_total
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item()
                _, predicted = outputs.max(1)
                val_total += labels.size(0)
                val_correct += predicted.eq(labels).sum().item()
        
        avg_val_loss = val_loss / len(val_loader)
        val_accuracy = 100. * val_correct / val_total
        
        # Get current learning rate
        current_lr = optimizer.param_groups[0]['lr']
        
        # Update history
        history['train_loss'].append(avg_train_loss)
        history['train_acc'].append(train_accuracy)
        history['val_loss'].append(avg_val_loss)
        history['val_acc'].append(val_accuracy)
        history['learning_rates'].append(current_lr)
        
        # Print progress
        print(f"Epoch [{epoch+1}/{num_epochs}]")
        print(f"  Train Loss: {avg_train_loss:.4f}, Train Acc: {train_accuracy:.2f}%")
        print(f"  Val Loss: {avg_val_loss:.4f}, Val Acc: {val_accuracy:.2f}%")
        print(f"  Learning Rate: {current_lr:.6f}")
        
        # Save best model based on validation loss
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            best_model_path = f"{save_dir}/best_model_epoch_{epoch+1}.pth"
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_loss': avg_val_loss,
                'val_acc': val_accuracy
            }, best_model_path)
            print(f"  ✓ Best model saved: {best_model_path}")
        
        # Save checkpoint every epoch
        checkpoint_path = f"{save_dir}/checkpoint_epoch_{epoch+1}.pth"
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_loss': avg_val_loss,
            'val_acc': val_accuracy
        }, checkpoint_path)
        
        print("-" * 70)
        
        # Update learning rate schedulers
        step_scheduler.step(epoch)
        plateau_scheduler.step(avg_val_loss)
    
    print(f"Training completed! Best validation loss: {best_val_loss:.4f}")
    print(f"Best model saved at: {best_model_path}")
    
    return history, best_model_path

In [80]:
# Evaluation function
def evaluate_model(model, test_loader, device='cpu'):
    """
    Evaluate the model on test data.
    
    Args:
        model: Trained model
        test_loader: DataLoader for test data
        device: Device to evaluate on
    
    Returns:
        Dictionary containing evaluation metrics
    """
    model = model.to(device)
    model.eval()
    
    all_predictions = []
    all_labels = []
    all_probabilities = []
    
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            outputs = model(images)
            probabilities = outputs.cpu().numpy()
            _, predicted = outputs.max(1)
            
            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.numpy())
            all_probabilities.extend(probabilities)
    
    all_predictions = np.array(all_predictions)
    all_labels = np.array(all_labels)
    all_probabilities = np.array(all_probabilities)
    
    # Calculate metrics
    accuracy = accuracy_score(all_labels, all_predictions)
    precision = precision_score(all_labels, all_predictions, average='binary')
    recall = recall_score(all_labels, all_predictions, average='binary')
    f1 = f1_score(all_labels, all_predictions, average='binary')
    
    # Confusion matrix
    cm = confusion_matrix(all_labels, all_predictions)
    
    results = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'confusion_matrix': cm,
        'predictions': all_predictions,
        'labels': all_labels,
        'probabilities': all_probabilities
    }
    
    print("Evaluation Results:")
    print(f"  Accuracy:  {accuracy:.4f}")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall:    {recall:.4f}")
    print(f"  F1-Score:  {f1:.4f}")
    print(f"\nConfusion Matrix:")
    print(cm)
    
    return results

In [81]:
# Visualization functions
def plot_training_history(history, save_path=None):
    """
    Plot training history including loss, accuracy, and learning rate.
    """
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    # Plot loss
    axes[0].plot(history['train_loss'], label='Train Loss', linewidth=2)
    axes[0].plot(history['val_loss'], label='Validation Loss', linewidth=2)
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Loss')
    axes[0].set_title('Training and Validation Loss')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Plot accuracy
    axes[1].plot(history['train_acc'], label='Train Accuracy', linewidth=2)
    axes[1].plot(history['val_acc'], label='Validation Accuracy', linewidth=2)
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Accuracy (%)')
    axes[1].set_title('Training and Validation Accuracy')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    # Plot learning rate
    axes[2].plot(history['learning_rates'], linewidth=2, color='green')
    axes[2].set_xlabel('Epoch')
    axes[2].set_ylabel('Learning Rate')
    axes[2].set_title('Learning Rate Schedule')
    axes[2].grid(True, alpha=0.3)
    axes[2].set_yscale('log')
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"Training history plot saved to {save_path}")
    
    plt.show()

def plot_confusion_matrix(cm, class_names=['Non-Asbestos', 'Asbestos'], save_path=None):
    """
    Plot confusion matrix as a heatmap.
    """
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=class_names, yticklabels=class_names,
                cbar_kws={'label': 'Count'})
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title('Confusion Matrix')
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"Confusion matrix saved to {save_path}")
    
    plt.show()

def visualize_predictions(images, labels, predictions, probabilities, num_samples=16):
    """
    Visualize sample predictions with their confidence scores.
    """
    num_samples = min(num_samples, len(images))
    indices = np.random.choice(len(images), num_samples, replace=False)
    
    rows = int(np.sqrt(num_samples))
    cols = int(np.ceil(num_samples / rows))
    
    fig, axes = plt.subplots(rows, cols, figsize=(cols*3, rows*3))
    axes = axes.flatten() if num_samples > 1 else [axes]
    
    for idx, ax in enumerate(axes):
        if idx >= num_samples:
            ax.axis('off')
            continue
        
        i = indices[idx]
        
        # Get image (convert from CHW to HWC and denormalize)
        img = images[i]
        if isinstance(img, torch.Tensor):
            img = img.permute(1, 2, 0).numpy()
        
        # Normalize to [0, 1] for display
        img = (img - img.min()) / (img.max() - img.min() + 1e-7)
        
        true_label = labels[i]
        pred_label = predictions[i]
        confidence = probabilities[i][pred_label] * 100
        
        ax.imshow(img)
        
        color = 'green' if true_label == pred_label else 'red'
        title = f"True: {'Asbestos' if true_label == 1 else 'Non-Asbestos'}\n"
        title += f"Pred: {'Asbestos' if pred_label == 1 else 'Non-Asbestos'} ({confidence:.1f}%)"
        
        ax.set_title(title, color=color, fontweight='bold')
        ax.axis('off')
    
    plt.tight_layout()
    plt.show()

In [82]:
# Load data from folder and CSV
def load_images_from_folder(csv_path='building_labels.csv', images_folder='building_images_labeled', image_size=(100, 100)):
    """
    Wczytuje obrazy z folderu i etykiety z CSV
    
    Args:
        csv_path: ścieżka do pliku CSV z etykietami
        images_folder: folder z obrazami
        image_size: docelowy rozmiar obrazów (100, 100)
    
    Returns:
        images: numpy array (N, 100, 100, 3)
        labels: numpy array (N,) z wartościami 0 lub 1
        filenames: lista nazw plików
    """
    # Wczytaj CSV
    df = pd.read_csv(csv_path)
    print(f"Wczytano {len(df)} wpisów z CSV")
    
    images = []
    labels = []
    filenames = []
    
    for idx, row in df.iterrows():
        filename = row['filename']
        has_asbestos = int(row['has_asbestos'])
        
        img_path = os.path.join(images_folder, filename)
        
        if os.path.exists(img_path):
            # Wczytaj i zmień rozmiar obrazu
            img = Image.open(img_path).convert('RGB')
            img = img.resize(image_size, Image.LANCZOS)
            img_array = np.array(img)
            
            images.append(img_array)
            labels.append(has_asbestos)
            filenames.append(filename)
    
    images = np.array(images, dtype=np.uint8)
    labels = np.array(labels, dtype=np.int64)
    
    print(f"✓ Wczytano {len(images)} obrazów")
    print(f"  Kształt obrazów: {images.shape}")
    print(f"  Kształt etykiet: {labels.shape}")
    
    # Sprawdź rozkład klas
    unique, counts = np.unique(labels, return_counts=True)
    print(f"\nRozkład klas:")
    for label, count in zip(unique, counts):
        label_name = 'Azbest' if label == 1 else 'Bez azbestu'
        print(f"  {label_name} ({label}): {count} ({count/len(labels)*100:.1f}%)")
    
    return images, labels, filenames

# Załaduj wszystkie dane
print("Ładowanie danych...")
images, labels, filenames = load_images_from_folder(
    csv_path='building_labels.csv',
    images_folder='building_images_labeled',
    image_size=(100, 100)
)

Ładowanie danych...
Wczytano 282 wpisów z CSV
✓ Wczytano 282 obrazów
  Kształt obrazów: (282, 100, 100, 3)
  Kształt etykiet: (282,)

Rozkład klas:
  Bez azbestu (0): 265 (94.0%)
  Azbest (1): 17 (6.0%)


In [83]:
# Podział na train/validation/test (80/10/10)
from sklearn.model_selection import train_test_split

print("\nPodział danych na train/validation/test...")

# Najpierw oddziel test set (10%)
train_val_images, test_images, train_val_labels, test_labels = train_test_split(
    images, labels, test_size=0.1, random_state=42, stratify=labels
)

# Następnie podziel pozostałe na train i validation (90% / 10% z pozostałych = 81% / 9% całości)
train_images, val_images, train_labels, val_labels = train_test_split(
    train_val_images, train_val_labels, test_size=0.111, random_state=42, stratify=train_val_labels
)

print(f"Train: {len(train_images)} ({len(train_images)/len(images)*100:.1f}%)")
print(f"Validation: {len(val_images)} ({len(val_images)/len(images)*100:.1f}%)")
print(f"Test: {len(test_images)} ({len(test_images)/len(images)*100:.1f}%)")

# Sprawdź rozkład klas w każdym zbiorze
for name, lbls in [('Train', train_labels), ('Validation', val_labels), ('Test', test_labels)]:
    unique, counts = np.unique(lbls, return_counts=True)
    print(f"\n{name} - rozkład klas:")
    for label, count in zip(unique, counts):
        print(f"  Klasa {label}: {count} ({count/len(lbls)*100:.1f}%)")

# Utwórz datasety
train_dataset = AsbestosDataset(train_images, train_labels, standardize=True)
val_dataset = AsbestosDataset(val_images, val_labels, standardize=True)
test_dataset = AsbestosDataset(test_images, test_labels, standardize=True)

# Utwórz data loadery
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=0)

print(f"\n✓ Loadery utworzone")
print(f"  Train batches: {len(train_loader)}")
print(f"  Val batches: {len(val_loader)}")
print(f"  Test batches: {len(test_loader)}")


Podział danych na train/validation/test...
Train: 224 (79.4%)
Validation: 29 (10.3%)
Test: 29 (10.3%)

Train - rozkład klas:
  Klasa 0: 211 (94.2%)
  Klasa 1: 13 (5.8%)

Validation - rozkład klas:
  Klasa 0: 27 (93.1%)
  Klasa 1: 2 (6.9%)

Test - rozkład klas:
  Klasa 0: 27 (93.1%)
  Klasa 1: 2 (6.9%)

✓ Loadery utworzone
  Train batches: 4
  Val batches: 1
  Test batches: 1


In [84]:
# Initialize model and check architecture
print("Initializing model...")
model = AsbestosDetectionCNN(num_classes=2)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

# Print model architecture summary
print("\nModel Architecture:")
print(model)

# Check if MPS is available
device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
print(f"\nUsing device: {device}")

# Test forward pass
test_input = torch.randn(2, 3, 100, 100).to(device)
model = model.to(device)
test_output = model(test_input)
print(f"\nTest forward pass - Input shape: {test_input.shape}, Output shape: {test_output.shape}")

Initializing model...
Total parameters: 54,298,754
Trainable parameters: 54,298,754

Model Architecture:
AsbestosDetectionCNN(
  (conv_block): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (inception1): InceptionBlock(
    (branch1): Sequential(
      (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (branch2): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (branch3): Sequential(
      (0): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, tra

In [85]:
# Train the model
print("Starting training...")
print("=" * 70)

history, best_model_path = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    num_epochs=128,
    learning_rate=0.0015,
    device=device,
    save_dir='./models'
)

print("\nTraining completed!")

Starting training...
Training on device: mps
Total epochs: 128, Batch size: 64
Epoch [1/128]
  Train Loss: 0.5419, Train Acc: 74.55%
  Val Loss: 0.3825, Val Acc: 93.10%
  Learning Rate: 0.001500
Epoch [1/128]
  Train Loss: 0.5419, Train Acc: 74.55%
  Val Loss: 0.3825, Val Acc: 93.10%
  Learning Rate: 0.001500
  ✓ Best model saved: ./models/best_model_epoch_1.pth
  ✓ Best model saved: ./models/best_model_epoch_1.pth
----------------------------------------------------------------------
----------------------------------------------------------------------
Epoch [2/128]
  Train Loss: 0.3745, Train Acc: 94.20%
  Val Loss: 0.3890, Val Acc: 93.10%
  Learning Rate: 0.001500
Epoch [2/128]
  Train Loss: 0.3745, Train Acc: 94.20%
  Val Loss: 0.3890, Val Acc: 93.10%
  Learning Rate: 0.001500
----------------------------------------------------------------------
----------------------------------------------------------------------
Epoch [3/128]
  Train Loss: 0.3725, Train Acc: 94.20%
  Val Loss:

KeyboardInterrupt: 

In [ ]:
# Plot training history
plot_training_history(history, save_path='./training_history.png')

In [ ]:
# Load best model for evaluation
print("Loading best model for evaluation...")
checkpoint = torch.load(best_model_path)
model.load_state_dict(checkpoint['model_state_dict'])
print(f"Loaded model from epoch {checkpoint['epoch']+1}")
print(f"Validation loss: {checkpoint['val_loss']:.4f}")
print(f"Validation accuracy: {checkpoint['val_acc']:.2f}%")

In [ ]:
# Evaluate on test set
print("Evaluating on test set...")
print("=" * 70)
test_results = evaluate_model(model, test_loader, device=device)

In [ ]:
# Plot confusion matrix
plot_confusion_matrix(test_results['confusion_matrix'], 
                     class_names=['Non-Asbestos', 'Asbestos'],
                     save_path='./confusion_matrix.png')

In [ ]:
# Visualize predictions on test samples
# Get test data for visualization
test_images_viz = test_images[:64]  # First 64 samples
test_labels_viz = test_labels[:64]

# Create a temporary dataset for visualization
viz_dataset = AsbestosDataset(test_images_viz, test_labels_viz, standardize=True)
viz_loader = DataLoader(viz_dataset, batch_size=64, shuffle=False)

# Get predictions
model.eval()
with torch.no_grad():
    for images, labels in viz_loader:
        images = images.to(device)
        outputs = model(images)
        probabilities = outputs.cpu().numpy()
        _, predictions = outputs.max(1)
        predictions = predictions.cpu().numpy()
        
        visualize_predictions(images.cpu(), labels.numpy(), predictions, probabilities, num_samples=16)
        break

In [ ]:
# Save final model for deployment
final_model_path = './models/asbestos_detection_final.pth'
torch.save({
    'model_state_dict': model.state_dict(),
    'model_architecture': 'AsbestosDetectionCNN',
    'input_size': (100, 100, 3),
    'num_classes': 2,
    'test_accuracy': test_results['accuracy'],
    'test_f1_score': test_results['f1_score']
}, final_model_path)

print(f"Final model saved to: {final_model_path}")
print(f"Test Accuracy: {test_results['accuracy']:.4f}")
print(f"Test F1-Score: {test_results['f1_score']:.4f}")

# Asbestos Roof Detection with CNN

## Model Architecture
This implementation is based on the paper's CNN architecture with inception-net feature extraction blocks. The model uses:

- **Input**: 100x100x3 RGB images
- **Initial Conv Block**: 64 filters, 3x3 kernel
- **4 Inception Blocks**: Each combining 1x1, 3x3, 5x5 convolutions and max pooling
- **Pooling & Dropout**: After each inception block (dropout rate 0.55)
- **2 Fully Connected Blocks**: 1024 neurons each with batch normalization and dropout (0.50)
- **Output**: Softmax layer for binary classification

## Training Configuration
- **Epochs**: 128
- **Batch Size**: 64
- **Initial Learning Rate**: 0.0015
- **Optimizer**: Adam
- **Loss Function**: Binary Cross-Entropy
- **LR Schedule**: Reduced by 15% every 10 epochs + plateau-based reduction
- **Standardization**: Feature-wise standardization applied to each image

## Key Features
1. Inception blocks enable multi-scale feature extraction
2. Feature-wise standardization handles variable lighting conditions
3. Aggressive dropout (0.55 spatial, 0.50 regular) prevents overfitting
4. Model checkpointing saves best model based on validation loss